# Ask GPT to predict loan risk based on financial record

In [1]:
import openai
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os


In [2]:
# get the API key from the json file
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [3]:
# import libraries - load and clean data
import pandas as pd
import openai
import json

# read in the credit_risk_dataset.json file into a pandas dataframe
whole_df = pd.read_csv('credit_risk_dataset.csv')
# cut it down to 200 rows for testing
df = whole_df[0:200]
# find features with missing values or NaN
#print(df.isnull().sum())

# clean the data replace NaN with -99.0
df = df.fillna(-99.0)
#re-order the columns so loan_status is last
cols = list(df.columns.values)
cols.pop(cols.index('loan_status'))
df = df[cols+['loan_status']]
# replace loan_status values 0 with paid and 1 with default
#df['loan_status'] = df['loan_status'].replace(0, 'low risk')
#df['loan_status'] = df['loan_status'].replace(1, 'high risk')
df.head()


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,0.59,Y,3,1
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0.10,N,2,0
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,0.57,N,3,1
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,0.53,N,2,1
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,0.55,Y,4,1


In [4]:
# consolidate all data columns under data and loan_status under status 
tdata = []
tstatus = []
for index, row in df.iterrows():
    loan_status = row['loan_status']
    # drop the loan_status column
    row = row.drop('loan_status')
    # convert row to dictionary
    row = row.to_dict()
    # convert dictionary to string
    data = json.dumps(row)
    # remove curly braces
    data = data.replace('{', '')
    data = data.replace('}', '')
    data = data.replace('"', '')
    tdata.append(data)
    tstatus.append(loan_status)



In [10]:
# create a pandas from tdata and tstatus as columns
df = pd.DataFrame({'data': tdata, 'loan_status': tstatus})
# loop over rows and print data
for index, row in df.iterrows():
    if index > 1:
        break
    print(row['data'])
   
 



person_age: 22, person_income: 59000, person_home_ownership: RENT, person_emp_length: 123.0, loan_intent: PERSONAL, loan_grade: D, loan_amnt: 35000, loan_int_rate: 16.02, loan_percent_income: 0.59, cb_person_default_on_file: Y, cb_person_cred_hist_length: 3
person_age: 21, person_income: 9600, person_home_ownership: OWN, person_emp_length: 5.0, loan_intent: EDUCATION, loan_grade: B, loan_amnt: 1000, loan_int_rate: 11.14, loan_percent_income: 0.1, cb_person_default_on_file: N, cb_person_cred_hist_length: 2


In [19]:
  # set the prompt
prompt = "You are a financial loan officer who has a low risk tolerance. You need to decide if the following person is a good candidate for a loan using the data to inform your decision. Here is their data: " + data + "}. In your response assign a risk level between 0 and 5 with 5 as the highest risk level. Deny if the risk level is above 2 or the loan_grade is C or worse.  Use the following format {decision: ,  risk_level: ,logic: }"

In [6]:
# Send data to OpenAI API along with prompt
import openai
import os

N = 8
def classifyPerson(data):
    # get the API key from the json file
    openai.api_key = os.environ.get('OPENAI_API_KEY')

    prompt = "You are a financial loan officer in France who has a low risk tolerance. You need to decide if the following person is a good candidate for a loan using the data to inform your decision. Here is their data: " + data + "}. In your response assign a risk level between 0 and 5 with 5 as the highest risk level. Deny if the risk level is above 2 or the loan_grade is C or worse.  Use the following format {decision: ,  risk_level: ,logic: }"
    # call the API
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.0,
    )

    # print the response
    print(response.choices[0].text)



In [ ]:
# loop over first N rows of df and pass data to GPT via classifyPerson
for index, row in df.iterrows():
    if index < N:
        data = row['data']
        classifyPerson(data)

In [ ]:
for index, row in df.iterrows():
    if index < N:
        data = row['loan_status']
        if data == 0:
            data = 'low risk'
        else:
            data = 'default'
        print("Actual Decision: "+ data)
